# HLY Saturation from Concentration

In [11]:
import pandas as pd
import seawater as sw
import numpy as np

In [12]:
def O2PercentSat(oxygen_conc=None, temperature=None, salinity=None, pressure=0):
    """
    # calculate oxygen saturation
    # Garcia and Gorden 1992 - from Seabird Derived Parameter Formulas
    
    """

    coefs = {
        "GG_A0": 2.00907,
        "GG_A1": 3.22014,
        "GG_A2": 4.0501,
        "GG_A3": 4.94457,
        "GG_A4": -0.256847,
        "GG_A5": 3.88767,
        "GG_B0": -0.00624523,
        "GG_B1": -0.00737614,
        "GG_B2": -0.010341,
        "GG_B3": -0.00817083,
        "GG_C0": -0.000000488682,
    }

    scaled_temp = np.log((298.15 - temperature) / (273.15 + temperature))

    Oxsol_pri = np.exp(
        coefs["GG_A0"]
        + coefs["GG_A1"] * scaled_temp
        + coefs["GG_A2"] * (scaled_temp) ** 2
        + coefs["GG_A3"] * (scaled_temp) ** 3
        + coefs["GG_A4"] * (scaled_temp) ** 4
        + coefs["GG_A5"] * (scaled_temp) ** 5
        + salinity
        * (
            coefs["GG_B0"]
            + coefs["GG_B1"] * scaled_temp
            + coefs["GG_B2"] * (scaled_temp) ** 2
            + coefs["GG_B3"] * (scaled_temp) ** 3
        )
        + coefs["GG_C0"] * (salinity) ** 2
    )

    # determine sigmatheta and convert Oxygen from micromoles/kg to ml/l
    # calculate new oxygen saturation percent using derived oxsol
    sigmatheta_pri = sw.eos80.pden(salinity, temperature, pressure)
    OxPerSat_pri = ((oxygen_conc * sigmatheta_pri / 44660) / Oxsol_pri) * 100.0

    # replace nan/1e35 with 1e35, >1e10
    try:
        OxPerSat_pri[oxygen_conc >= 1e30] = np.nan
    except:
        if OxPerSat_pri >= 1e30:
            OxPerSat_pri = np.nan

    return OxPerSat_pri

In [13]:
df = pd.read_excel('/Users/bell/ecoraid/2019/CTDcasts/hly1901/final_data/csv/he1901_merged_final.xlsx',skiprows=31,sheet_name=0)

In [14]:
df.sample()

,Cruise,Cast,Latitude,Longitude,Date Time,P_4,S_42,S_41,Fch_906,T_28,...,PO4_186,NO2_184,NO3_182,NH4_189,Notes,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30
16996,he18897,98,72.4153,-155.5032,2019-08-18 02:09:00,23,29.96757,29.96226,0.6734,0.497,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
df['CTDOST_4221'] = O2PercentSat(oxygen_conc=df.CTDOXY_4220, temperature=df.T2_35, salinity=df.S_42, pressure=df.P_4)
df['OST_62'] = O2PercentSat(oxygen_conc=df.O_65, temperature=df.T_28, salinity=df.S_41, pressure=df.P_4)

In [18]:
df.to_csv('/Users/bell/ecoraid/2019/CTDcasts/hly1901/final_data/csv/he1901_merged_final.woxy.csv')